In [2]:
import gensim
# Load Google's pre-trained Word2Vec model.
# Downloadable from: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
model.similarity('Artificial_Intelligence','Robotics')

0.42267702789777861

In [121]:
# Load results of Synset and Fusion(a=2)
import pandas as pd
synset_df = pd.read_csv('../stst/per_article_eval/baseline_eval.csv')
fusion2_df = pd.read_csv('../stst/per_article_eval/evaluation.csv')
synset_recommendation = synset_df['y_predict']
fusion2_recommendation = fusion2_df['y_predict']
synset_test = synset_df['y_test']
fusion_test = fusion2_df['y_test']
fusion3_df = pd.read_csv('../stst_r4/per_article_eval/evaluation.csv')
fusion1_df = pd.read_csv('../stst_n1/per_article_eval/evaluation.csv')
fusion4_df = pd.read_csv('../stst_n4/per_article_eval/evaluation.csv')
fusion1_recommendation = fusion1_df['y_predict']
fusion1_test = fusion1_df['y_test']
fusion3_recommendation = fusion3_df['y_predict']
fusion3_test = fusion3_df['y_test']
fusion4_recommendation = fusion4_df['y_predict']
fusion4_test = fusion4_df['y_test']

In [127]:
def clean_topic(topic):
    topic = topic.replace(' ','')
    topic = topic.replace('\'','')
    topic = topic.lstrip('\'')
    topic = topic.lstrip('\"')
    topic = topic.lstrip('\'')
    topic = topic.rstrip('\'')
    topic = topic.rstrip('\"')
    topic = topic.rstrip('\'')
    return topic

def unexpectedness(res_recommendation, test):
    global_counter = 0
    global_sim = 0.0
    for i, recommendations in enumerate(res_recommendation):
        recommendations.replace(' ','')
        test[i].replace(' ','')
        recommendations = recommendations[1:-1].split(',')
        expectations = test[i][1:-1].split(',')
        if not recommendations == expectations:
            similarity = 0.0
            counter = 0
            for topic in recommendations:
                topic = clean_topic(topic)
                expectation = clean_topic(expectations[0])
                if expectation in ['Information_Systems', 'international_relations', 'Respiratory_System']:
                    break
                if topic in ['Information_Systems', 'international_relations', 'Respiratory_System']:
                    break
                similarity += model.similarity(topic,expectation)
                #print model.similarity(topic,expectation)
                counter += 1
            if counter > 0:
                avg_similarity = similarity / counter
                #print i, recommendations, expectation, avg_similarity, similarity, counter
                global_sim += avg_similarity
                global_counter += 1
    return 1.0 - (global_sim / global_counter)

In [128]:
print 'Synset unexpectedness = ', unexpectedness(synset_recommendation, synset_test)
print 'Fusion1 unexpectedness = ', unexpectedness(fusion1_recommendation, fusion1_test)
print 'Fusion2 unexpectedness = ', unexpectedness(fusion2_recommendation, fusion_test)
print 'Fusion3 unexpectedness = ', unexpectedness(fusion3_recommendation, fusion3_test)
print 'Fusion4 unexpectedness = ', unexpectedness(fusion4_recommendation, fusion4_test)

Synset unexpectedness =  0.587024405206
Fusion1 unexpectedness =  0.572921174281
Fusion2 unexpectedness =  0.558625720562
Fusion3 unexpectedness =  0.519269696642
Fusion4 unexpectedness =  0.509035913438


In [123]:
def ind_diversity(res_recommendation, test):
    global_counter = 0
    global_sim = 0.0
    for i, recommendations in enumerate(res_recommendation):
        recommendations.replace(' ','')
        test[i].replace(' ','')
        recommendations = recommendations[1:-1].split(',')
        expectations = test[i][1:-1].split(',')
        if not (recommendations == expectations or len(recommendations) == 1):
            similarity = 0.0
            counter = 0
            for i in range(len(recommendations)):
                topic = recommendations[i]
                topic_a = clean_topic(topic)
                if topic_a in ['Information_Systems', 'international_relations', 'Respiratory_System']:
                    break
                for j in range(i,len(recommendations)):
                    topic = recommendations[j]
                    topic_b = clean_topic(topic)
                    if topic_b in ['Information_Systems', 'international_relations', 'Respiratory_System']:
                        break
                    similarity += model.similarity(topic_a, topic_b)
                    #print model.similarity(topic,expectation)
                    counter += 1
                if counter > 0:
                    avg_similarity = similarity / counter
                    #print i, recommendations, expectation, avg_similarity, similarity, counter
                    global_sim += avg_similarity
                    global_counter += 1
    return 1.0 - (global_sim / global_counter)

In [125]:
print 'Synset diversity = ', ind_diversity(synset_recommendation, synset_test)
print 'Fusion1 diversity = ', ind_diversity(fusion1_recommendation, fusion1_test)
print 'Fusion2 diversity = ', ind_diversity(fusion2_recommendation, fusion_test)
print 'Fusion3 diversity = ', ind_diversity(fusion3_recommendation, fusion3_test)
print 'Fusion4 diversity = ', ind_diversity(fusion4_recommendation, fusion4_test)

Synset diversity =  0.264854842801
Fusion1 diversity =  0.247195072259
Fusion2 diversity =  0.258550169678
Fusion3 diversity =  0.267989073312
Fusion4 diversity =  0.277376857984
